In [ ]:
import json
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, TrainerCallback

# Custom Callback for logging
class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(logs)

# Load the datasets
with open('train.json', 'r') as file:
    train_dataset = json.load(file)
train_dataset = Dataset.from_list(train_dataset)

with open('test.json', 'r') as file:
    test_dataset = json.load(file)
test_dataset = Dataset.from_list(test_dataset)

# Model and tokenizer
model_name = "persiannlp/mt5-small-parsinlu-translation_en_fa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Preprocessing function
def preprocess(data):
    word = data["word"]
    sentence = data["sentence"]
    model_input = tokenizer(word, max_length=32, truncation=True, padding="max_length")
    labels = tokenizer(sentence, max_length=128, truncation=True, padding="max_length").input_ids
    model_input['labels'] = labels
    return model_input

# Tokenize datasets
tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_test = test_dataset.map(preprocess, batched=True)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v_"],
    lora_dropout=0.05,
    bias="none"
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/temp/result",
    evaluation_strategy="steps",
    eval_steps=50,
    logging_strategy="steps",
    logging_steps=50,  # Log every 50 steps
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    logging_dir="/content/drive/MyDrive/temp/logs",
    report_to="tensorboard"
)

# Trainer with custom callback
trainer = Seq2SeqTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[LoggingCallback()]  # Add the custom logging callback
)

# Start training
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
50,32.625400,No log


{'loss': 32.6254, 'grad_norm': 3.787834644317627, 'learning_rate': 0.0, 'epoch': 2.0}
{'eval_runtime': 124.5565, 'eval_samples_per_second': 1.028, 'eval_steps_per_second': 0.064, 'epoch': 2.0}
{'train_runtime': 1728.2189, 'train_samples_per_second': 0.462, 'train_steps_per_second': 0.029, 'total_flos': 26397748887552.0, 'train_loss': 32.625390625, 'epoch': 2.0}


{'eval_runtime': 121.7202, 'eval_samples_per_second': 1.052, 'eval_steps_per_second': 0.066, 'epoch': 2.0}
{'eval_runtime': 121.7202, 'eval_samples_per_second': 1.052, 'eval_steps_per_second': 0.066, 'epoch': 2.0}


In [ ]:
model.save_pretrained("/content/drive/MyDrive/temp/model")
tokenizer.save_pretrained("/content/drive/MyDrive/temp/tokenizer")

# ذخیره مدل LoRA
lora_model.save_pretrained("/content/drive/MyDrive/temp/model")

In [ ]:
from google.colab import files
import shutil

# فشرده‌سازی پوشه مدل
shutil.make_archive('/content/model', 'zip', '/content/drive/MyDrive/temp/model')
shutil.make_archive('/content/tokenizer', 'zip', '/content/drive/MyDrive/temp/tokenizer')

# دانلود فایل‌های فشرده شده
files.download('/content/model.zip')
files.download('/content/tokenizer.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>